In [1]:
# DisInfo Project Analysis V1 201804
# (based on OpenWPM-scan of Haye list on 201804 & earlier lightbeam-scan by Hossein on 201803)
# update 20180524: export as matrix for Amir/Marc analysis and for Elsa/Haye check

import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import matplotlib
from collections import Counter
from urllib.parse import urlparse

matplotlib.style.use('ggplot')
#DPI = 120  # used for graphs. check at: http://www.infobyip.com/detectmonitordpi.php

In [2]:
# Helper function to remove the http, https and extract domains from site_url

from tldextract import extract as tldextract  # pip install tldextract

def strip_site(site):
    """Removes leading http:// or https:// and trailing '/'"""
    if site is None:
        return None
    site = site.lower().strip()
    if site.startswith('http'):
        site = site.replace('https://', '').replace('http://', '')
    if site.startswith('www.'):
        site = site[4:]
    if site.endswith('/'):
        site = site[:-1]
    return site

def extract_domain(site):
    """Returns domain+tld from a full domain"""
    site = strip_site(site)  # just to be sure
    ext = tldextract(site)
    ret = ".".join([ss for ss in ext[-2:] if ss])
    # In case of gov.ie, or IPs, our 'ret' works better than 'ext.registered_domain'
    return ret

In [3]:
# Q 1- which sites broken to filter
#    all broken, of course (success=0). 
#    also: the ones that returned 404 / plesk / etc? -- would need check

dbcon = sqlite3.connect("../out/disinfo.20180417.KEEP/crawl-data.sqlite")

# sqlite reference: https://sqlite.org/lang_corefunc.html
sites = pd.read_sql_query("""SELECT s.visit_id, 
                            site_url as site, 
                            bool_success, 
                            group_concat(response_status) as response_status, 
                            group_concat(location) as redirect, 
                            count(*) as response_count
                          FROM site_visits s 
                          LEFT JOIN CrawlHistory h
                               ON s.site_url=h.arguments AND h.command='GET'
                          LEFT JOIN http_responses r
                               ON s.visit_id=r.visit_id AND LOWER(RTRIM(s.site_url))=RTRIM(r.url, '/')
                          GROUP BY s.visit_id""", 
                          dbcon)
       
sites["site"] = sites["site"].apply(strip_site)
sites["redirect"] = sites["redirect"].apply(strip_site)
sites = sites[sites["site"]!= "geoip.hidemyass.com"]  # remove our test/vpn site
print(sites.groupby("bool_success").site.count())  
# bool-success 17:0=non-existant, rest:ok=1 => 2:-1=crash? (zaplog.nl, nieuws.nl)

print("check RCOUNT>2:", sites[sites.response_count>1].site)   # koffiepauze.com -- tagged as non-functional
t = sites[(sites.response_count==1)&(sites.bool_success==1)].copy()
print("check 400/500 errors:", t[t.response_status.astype(int)>=400].site)  # zonnewinde.be, also non-functional
del sites["response_count"]  # not needed anymore

# ADD site type
st = pd.read_csv("../indata/disinfo-tracker-site-list-201804b.csv")  
# 20180524: 'b' added, think tld/rank cols delted, screenshot_check renamed to: site-functional
#print(st.columns)
st["site"] = st["site"].apply(str.lower).apply(str.strip)
del st["source"]

# combining some types -- to check all (agh, 4 where wrongly classified by me, fixed)
st.loc[st["site_type"]=="NL-opinion-islamic", "site_type"] = "NL-opinion"
st.loc[st["site_type"]=="NL-politics-DH", "site_type"] = "NL-pparties"
st.loc[st["site_type"]=="NL-politics", "site_type"] = "NL-pparties"
st.loc[st["site_type"]=="NL-valse-nieuws", "site_type"] = "NL-clickbait"  

sites = sites.set_index("site").join(st.set_index("site"))
#print(sites.groupby(["site_type", "bool_success"]).count())  # note: the unsuccess are NL-parties


# for now, manually fixing 'not found'/'redirects'
# CHECKED: sites[(sites.screenshot_check==0)&(sites.bool_success==1)] 
#   -- mostly clickbaits that have been since removed; plus tmgonlinemedia/dds/pvvdenhaag
# CHECKED: sites[(sites.screenshot_check==1)&(sites.bool_success!=1)]  == None

# thus remove unsuccessful sites
sites = sites[(sites.bool_success==1)&(sites["site-functional"]==1)]  
print("\n FINAL", len(sites)) 

#print(sites.groupby(["site_type]).count())  # final


bool_success
-1      2
 0     17
 1    148
Name: site, dtype: int64
check RCOUNT>2: 42    koffiepauze.com
Name: site, dtype: object
check 400/500 errors: 35    zonnewind.be
Name: site, dtype: object

 FINAL 134


In [4]:
# 20180524 REDIRECTs => TODO: ELSA!
sites.reset_index(inplace=True)
mark_del = set()

for i, r in sites.iterrows():
    if r.redirect != r.site and r.redirect:        
        #print(r.site, "=>", r.redirect, "*" if r.redirect in sites.index else "")
        
        
        if r.redirect in set(sites.site):
            # duplicate, we can delete it
            print('deleting duplicate', r.site, '(', r.redirect, ')')
            mark_del.add(r.site)
        else:
            # otherwise rename it. this is more accurate. see below stats
            sites.loc[sites.site==r.site, 'site'] = r.redirect  
            print('rename', r.site, '=>', r.redirect)
        

sites = sites[~sites.site.isin(mark_del)]
print(len(sites))
sites.set_index('site', inplace=True)
        
# nothing super interesting here :)        
#         dagelijksfilmpje.nl => coinbids.io
#         hetdelenwaard.nl => hetdelenwaard.net
#         vk.nl => volkskrant.nl *
#         joop.nl => joop.bnnvara.nl
#         d66denhaag.nl => denhaag.d66.nl *
#         fvd.nl => forumvoordemocratie.nl

# UPON FURTHER INVESTIATGION, FOR D66 & VK & FVD & JOOP THE REDIRECTED DOMAINS ARE CORRECTER
# (in terms of 3rd parties) -- so let's make that change

# for s in ['vk.nl', 'volkskrant.nl', 'd66denhaag.nl', 'denhaag.d66.nl', 'fvd.nl', 'joop.nl']:
#     ct = cookies[cookies.site==s]            
#     jt = js_calls[js_calls.site==s]    
#     print(s, set(ct.chost), set(jt.jhost))
    



rename dagelijksfilmpje.nl => coinbids.io
rename hetdelenwaard.nl => hetdelenwaard.net
rename joop.nl => joop.bnnvara.nl
deleting duplicate vk.nl ( volkskrant.nl )
rename fvd.nl => forumvoordemocratie.nl
deleting duplicate d66denhaag.nl ( denhaag.d66.nl )
132


In [6]:
#overlap with hossein list

hossein = pd.read_csv("../indata/disinfo ThirdPartySites-Hossein 20180323.csv") 
hossein["site"] = hossein["site"].apply(lambda x: x.replace("www.", ""))
hossein = hossein.filter(["site", "thirdparties"])

for s in hossein["site"]:
    if s not in sites.index:
        print(s)
        
# some missing -- to add? 

dds.nl
trendbuzz.nu
viraltube.nl
nidarotterdam.nl
partijvoordedieren.nl
sgp.nl
axed.nl
viraalvandaag.com


In [7]:
cookies = pd.read_sql_query("""SELECT site_url as site, host as chost, name, value, 
                                      is_session, change, expiry, creationTime
                            FROM site_visits s 
                            LEFT JOIN javascript_cookies c 
                            ON s.visit_id=c.visit_id ORDER BY s.visit_id""", 
                            dbcon)

print("cookies:", len(cookies))

# remove null cookies  (q: what are they?)
cookies['site'] = cookies["site"].astype(str).apply(strip_site)
cookie_null_sites = set(cookies[cookies["chost"].isnull()].site)
cookies['chost'] = cookies["chost"].astype(str).apply(extract_domain)
cookies = cookies[(~cookies["name"].isnull())]

# compare site-domain and cookie domain;  keep only 3rd party cookies
comparison_1pd = cookies["chost"] == cookies["site"].apply(extract_domain)
cookies = cookies[~comparison_1pd]
print("removing 1st-parties:", len(cookies))

# if a cookie is changed, keep only last; if deleted, remove. 

# cookies_US_unique=pd.DataFrame(data=US.drop_duplicates(subset=['name','site_url'],keep='last'))
# cookies_US_unique.head()
# cookies_US_unique.shape

# creates a combo key with ~ for easier checks (does a sanity check that ~ isn't used first)
assert not [ix for ix, c in cookies.iterrows() if "~" in c["site"] or "~" in c["chost"]]#  or "~" in c["name"]]
cookies["combo"] = cookies["site"] + "~" + cookies["chost"] + "~" + cookies["name"]
tmp_seen = set()
tmp_added = list(cookies[cookies.change=='added']['combo'])  
for ix in reversed(cookies.index):
    sc = cookies.loc[ix, "combo"]
    if sc not in tmp_seen:
        tmp_seen.add(sc)
        assert sc in tmp_added  # sanity: make sure any changed/deleted/added cookie was added first
    else:
        cookies.loc[ix, "combo"] = None  # cookie later seen, mark earlier for deletion
cookies = cookies[~cookies["combo"].isnull()]
del cookies["combo"]
print("removing duplicates:", len(cookies))

# convert dates and get only cookies that expire in more than 30 days.
# (this also takes out the 340 is_session==1)
cookies.loc[:,"expiry"] = cookies["expiry"].astype('datetime64[ns]') 
cookies.loc[:,"creationTime"] = cookies["creationTime"].astype('datetime64[ns]')
cookies["expiry_life"] = cookies.loc[:,"expiry"] - cookies.loc[:,"creationTime"] 
cookies = cookies[cookies["expiry_life"] > pd.Timedelta(30,unit='d')]
print("removing short-lived:", len(cookies))

# remove unsuccessful visits -- not possibly not all where unsuccseful; anyway click-bait group, don't care
print(set(cookies[~cookies["site"].isin(sites.index)]["site"]))
    # removing: {'geoip.hidemyass.com', 'nieuws.nl', 'zaplog.nl'}
cookies = cookies[cookies["site"].isin(sites.index)]
print("removing unsuccseful:", len(cookies))

# simplify dataframe
cookies["cookie"] = cookies["chost"] + "~" + cookies["name"]
del cookies["is_session"], cookies["change"], cookies["expiry"], cookies["creationTime"], cookies["name"]
cookies.head()


cookies: 5880


unable to cache TLDs in file /opt/miniconda3/lib/python3.6/site-packages/tldextract/.tld_set: [Errno 13] Permission denied: '/opt/miniconda3/lib/python3.6/site-packages/tldextract/.tld_set'


removing 1st-parties: 4987
removing duplicates: 2321
removing short-lived: 1615
{'nieuws.nl', 'hetdelenwaard.nl', 'fvd.nl', 'viraltube.nl', 'geoip.hidemyass.com', 'joop.nl', 'vk.nl', 'zaplog.nl', 'viraltrend.nl', 'd66denhaag.nl', 'koffiepauze.com', 'copernicusmarkpeeters.skynetblogs.be', 'dagelijksfilmpje.nl'}
removing unsuccseful: 1338


,site,chost,value,expiry_life,cookie
27,ahealthyme.nl,doubleclick.net,AHWqTUm7qEUguv-R-wXnwyO4lqOMn1zD-_dMkJxqrVB6mG...,390 days,doubleclick.net~IDE
34,apost.com,adnxs.com,ChgIldY8EAoYASABKAEw4a7Z1gU4AUABSAEQ4a7Z1gUYAA..,90 days,adnxs.com~icu
36,apost.com,onesignal.com,d3b207893efd67bfe1ec1c5f8738c7dc21523996511,365 days,onesignal.com~__cfduid
46,apost.com,taboola.com,CwsIIBDlgAoMCwgkEOWACgwLCC0Q5YAKDAsIJxDlgAoMDBMU,365 days,taboola.com~stpt
48,apost.com,taboola.com,CAETCMquPxABFA,365 days,taboola.com~t_vpub


In [8]:
# get http requests (includes iframes, static resources, js-calls) 
# => instead I use the JS-calls group (below). that's a bit more conservative. and need to check if it contains FB

# http_reqs = pd.read_sql_query("""SELECT site_url as site, url as req_url, url as rhost
#                                 FROM site_visits s LEFT JOIN http_requests r
#                                 ON s.visit_id=r.visit_id 
#                                 ORDER BY s.visit_id""", 
#                                dbcon)
# http_reqs['site'] = http_reqs["site"].astype(str).apply(strip_site)
# http_reqs['rhost'] = http_reqs["rhost"].astype(str).apply(extract_domain)

# # Q: filter at least same domain
# comparison_1pd = http_reqs["rhost"] == http_reqs["site"].apply(extract_domain)
# http_reqs = http_reqs[~comparison_1pd]
# print(len(http_reqs))  

# # also filter most basic static resources 
# # note, images (.png, .jpg, .jpeg, .svg), as well as .js/.html/.... can be/call trackers
# comparison_static = http_reqs.req_url.apply(lambda x: x.endswith('.css') or x.endswith('.ttf'))
# http_reqs = http_reqs[~comparison_static]
# print('removing static', len(http_reqs))  

# http_reqs['rcompany'] = http_reqs['rhost']
# for h, c in map_domain_company.items():
#     http_reqs.loc[http_reqs.rhost==h, 'rcompany'] = c

# http_reqs.groupby('rcompany').site.nunique().sort_values(ascending=False)[:15]  # has g-analytics, etc  


In [9]:
map_domain_company = {"google-analytics.com": "GOOGLE",  
                     # Q: perhaps separate google-static & not? (check STEVE's paper)
                    "googleapis.com": "GOOGLE",        
                    "gstatic.com": "GOOGLE",
                    "doubleclick.net": "GOOGLE",
                    "google.com": "GOOGLE",
                    "google.nl": "GOOGLE",
                    "googlesyndication.com": "GOOGLE",
                    "googletagmanager.com": "GOOGLE",
                    "googleadservices.com": "GOOGLE",  # Q: why so many google domains??
                    "youtube.com": "GOOGLE",
                    "ytimg.com": "GOOGLE",  # (i could script WHOIS on all of these :)
                    "googletagservices.com": "GOOGLE",
                     "facebook.com": "FACEBOOK",
                    "facebook.net": "FACEBOOK",
                    "fbcdn.net": "FACEBOOK",                    
                    "instagram.com": "FACEBOOK",
                    "twimg.com": "twitter.com",
                    "dummy file": None,  # for js-urls                    
                    # cloudflare.com, scorecardresearch.com, bidswitch.net...
                    # cloudfront.net=amazon?
                    } 

In [10]:
# jhost NOT needed, all of these appear in the rhost. 
# BUT maybe a btit more conservative re js vs static elements (e.g. css)
js_calls = pd.read_sql_query("""SELECT site_url as site, script_url as js_url, script_url as jhost
                                FROM site_visits s LEFT JOIN javascript j
                                ON s.visit_id=j.visit_id 
                                WHERE script_url is not null
                                ORDER BY s.visit_id""", dbcon)
js_calls['site'] = js_calls["site"].astype(str).apply(strip_site)
js_calls['jhost'] = js_calls["jhost"].astype(str).apply(extract_domain)
js_calls.groupby('jhost').site.nunique().sort_values(ascending=False)[:10]  

# Q: filter same domain
comparison_1pd = js_calls["jhost"] == js_calls["site"].apply(extract_domain)
js_calls = js_calls[~comparison_1pd]

# remove 'dummy file' -- not sure what it is
js_calls = js_calls[js_calls.js_url!='dummy file']  # approx 400 call (1.5%)

# STRIP QUERY from js
js_calls.js_url = js_calls.js_url.apply(lambda x: urlparse(x).netloc + urlparse(x).path)  

# # map to company when known
js_calls['jcompany'] = js_calls['jhost']
for h, c in map_domain_company.items():
    js_calls.loc[js_calls.jhost==h, 'jcompany'] = c

js_calls.groupby('jcompany').site.nunique().sort_values(ascending=False)[:15]  # has g-analytics, etc  


jcompany
GOOGLE                   118
FACEBOOK                  47
cloudflare.com            19
scorecardresearch.com     18
wp.com                    17
twitter.com               17
gravatar.com              13
taboola.com               12
cloudfront.net            11
rootads.nl                10
adnxs.com                  7
statcounter.com            7
chartbeat.com              6
sharethis.com              5
sprinklecontent.com        5
Name: site, dtype: int64

In [11]:
# NOW: top 3rd parties overall (domain)

# trackers = cookies.groupby('chost').site.count()  # .count() = TOTAL cookies 
# print(trackers.sort_values(ascending=False)[:5])  # 

cookies['ccompany'] = cookies['chost']
# map_chost_company = {
#     "doubleclick.net": "GOOGLE",
#     "youtube.com": "GOOGLE",
# }
for h, c in map_domain_company.items():
    cookies.loc[cookies.chost==h, 'ccompany'] = c
    
print(cookies.groupby('ccompany').site.nunique().sort_values(ascending=False)[:10]) # .nunique = one per domain
print(cookies.groupby('ccompany').site.count().sort_values(ascending=False)[:5])

# cookies[cookies.chost=="360yield.com"].groupby('chost').site.count()  # total cookies of this type
# cookies[cookies.chost=="360yield.com"].groupby('chost').site.nunique()  # on how many sites

# (combine groups, e.g. youtube + doubleclick same, manually => reduce overlap to calculate percentages )
# (could give output as pie-chart too)

ccompany
GOOGLE                   57
scorecardresearch.com    23
bidswitch.net            18
adnxs.com                18
adsrvr.org               18
FACEBOOK                 12
mathtag.com              11
taboola.com              11
rubiconproject.com       10
everesttech.net           9
Name: site, dtype: int64
ccompany
GOOGLE                   88
bidswitch.net            54
scorecardresearch.com    46
taboola.com              43
adnxs.com                39
Name: site, dtype: int64


In [12]:
ct_ = cookies.filter(["site", "ccompany"])
ct_.columns = ["site", "tracker"]
jt_ = js_calls.filter(["site", "jcompany"])
jt_.columns = ["site", "tracker"]
combo = ct_.append(jt_)

sum(combo.groupby('tracker').site.nunique() >= 15)
#, len(set(cookies.ccompany))

10

In [13]:
#sites[sites.site_type.isnull()]
#sum(cookies.is_session==1)

In [14]:
# trackers per (site) category
mm = set()

for t in ["nieuws", "opinion", "clickbait", "pparties"]:
    t = "NL-" + t
    ct = cookies.join(sites, on="site")
    ct = ct[ct["site_type"]==t]
        
    jt = js_calls.join(sites, on="site")
    jt = jt[jt["site_type"]==t]
    
    #rt = http_reqs.join(sites, on="site")
    #rt = rt[rt["site_type"]==t]    
    # assert len(set(rt.site)) - len(set(jt.site)) <= 3  # we are using the more conservative JT 
        
    ct_ = ct.filter(["site", "ccompany"])
    ct_.columns = ["site", "tracker"]
    jt_ = jt.filter(["site", "jcompany"])
    jt_.columns = ["site", "tracker"]
    combo = ct_.append(jt_)
    have_g = set(combo[combo["tracker"]=="GOOGLE"].site)
    have_fb = set(combo[combo["tracker"]=="FACEBOOK"].site)
    med_t = combo.groupby("site").tracker.nunique().median()
    
    
    print(t, len(set(ct.site)), "/", len(set(jt.site)),
          "of", len(sites[sites["site_type"]==t]),
          "=>", len(have_g), len(have_fb), med_t)
    
    m = set(st.index) - set(ct.site)
    #print(m, m & cookie_null_sites)
    mm |= m
        
# MADE A NICE TABLE BASED ON THIS FOR REPORT

# NOTE 20180524
# removing the 'dummy file' group we get 1-less js for news & pparty

# with rename/removal of 2, we have:
# 14+1/18+2 from 21+2
# 18+1/22+1 from 25+0
# 53+2/56+2 from 61+0
# 7+2/18+3 from 25+1

NL-nieuws 14 / 18 of 21 => 14 2 3.0
NL-opinion 18 / 22 of 25 => 20 10 3.5
NL-clickbait 53 / 56 of 61 => 55 32 5.5
NL-pparties 7 / 18 of 25 => 15 6 2.0


In [15]:
# TODO:    
# - CHECKED ALL 36, ALL CRAWLED SUCCESSFULY. (i.e. this is different from 18 that didn't load)
#    A BUNCH HAVE COOKIEWALLS, others unclear 
#    what disntiguishes the 'nullgroup' list? -- I cannot tell any difference
#    could it be a bug?    
# print(m, m & cookie_null_sites)
# mm = [(sites.loc[s, "visit_id"], s) for s in mm]  
# sorted(mm, key=lambda x:x[0])


In [16]:
# 20180524. This is the output I'll craete:

# two columns: website, our category. 
#              (REDIRECTED TO -- remove *, others keep original perhaps?)
#        
# other columns: trackers-http, trackers-js  -- based on domain.

# STILL open (method) QUESTIONS:
# - cookiewalls? => the new random clicks will find 
# - (check list -- are we say remivng third party calls from VK to volkskrant.com? => fixeD)
# - (what explains null cookies? cookiewalls? crashes?)
# - Q ROB/STEVE/ELSA: also what if cookies have same name/values (long strings) -- on different sites?

# additional file saying who tracking domains are  => TO COMPLETE ELSA
# - need some taxonomy (category of 3rd parties) to understand who/why they are used 
#   (e.g. ad business? other tracker? google owned like YT/DC?)
# - will be completed by elsa (manually; also info on domains;)
# - (this can also be checked agains tghostery list)


# IMPORTANT CHECK
# - does it include FACEBOOK LIKE BUTTON?? (or sth else)
# - why so many GOGOLE domains. in particular: GOOGLE static? => SEE: steve's paper
#   - google analytics cookie: ahealthyme.nl~_gid  -- 1st party??? -- how???
#     (https://developers.google.com/analytics/devguides/collection/analyticsjs/cookie-usage)
#   - doubleclick.net short-lived cookies = ??


# SITE LIST/CATEGORY => for Haye: 
# - manually check/confirm site categories & completeness 
# - are these missing? a few are at least redirecting, i think. rest perhaps need to be indexed
#   joop.bnnvara.nl, forumvoordemocratie.nl, nidarotterdam.nl, partijvoordedieren.nl, sgp.nl, axed.nl, viraalvandaag.com
# - also what thomas found


all_cts = set()
all_jts = set()

msites = sites.reset_index().sort_values(by=['site_type', 'site'])
msites = msites[~msites.site.isin(['vk.nl', 'd66denhaag.nl'])]  # FIX: we don't want the redirects

for i, r in msites.iterrows():   
    ct = cookies[cookies.site==r.site]            
    jt = js_calls[js_calls.site==r.site]    
    all_cts |= set(ct.chost.apply(lambda x: x+'/c'))
    all_jts |= set(jt.jhost.apply(lambda x: x+'/j'))    

    
# 63 in both: len(all_cts & all_jts); 198 cookies 135 javascript
#print(len(sites), len(all_ctjt))    
mcols = ["category"] + sorted(all_cts | all_jts)



matrix = pd.DataFrame(columns=mcols, index=msites.site)
#print(mcols, matrix.index)

for s in matrix.index:
    matrix.loc[s,'category'] = sites.loc[s, 'site_type']

    
for i, r in msites.iterrows():   
    print(r.site, end=' => ')
    n_c, n_j = 0, 0
    
    # TODO: this innter loop must have some simpler matrix way in pandas :)
    
    for c in all_cts:
        ct = cookies[(cookies.site==r.site)&(cookies.chost==c[:-2])]
        matrix.loc[r.site, c] = len(ct)
        n_c += len(ct)
        
    for j in all_jts:
        jt = js_calls[(js_calls.site==r.site)&(js_calls.jhost==j[:-2])]
        matrix.loc[r.site, j] = len(set(jt.js_url))
        n_j += len(set(jt.js_url))
        #for jj in set(jt.js_url):  n_jj += 1 if jj.endswith(".js") else 0
            
    print(n_c, n_j)
    


ahealthyme.nl => 1 13
apost.com => 135 25
belfort-group.eu => 0 0
bestgezond.nl => 6 9
bewustnieuws.nl => 0 0
blikopnosjournaal.blogspot.nl => 4 23
boinnk.nl => 6 7
brekendnieuws.nl => 14 17
coinbids.io => 0 0
dagelijks.nu => 2 14
delangemars.nl => 3 16
destillewaarheid.nl => 1 7
earth-matters.nl => 1 10
echtekrant.be => 3 2
ellaster.nl => 0 4
gewoon-nieuws.nl => 9 20
healthbytes.me => 6 23
healthunity.pw => 11 17
healthwatch.nu => 21 33
hetdelenwaard.net => 0 0
hln.be => 49 21
jdreport.com => 6 28
journaalflash.wordpress.com => 65 13
klokkenluideronline.is => 6 14
langleveeuropa.nl => 0 0
leeftips.nl => 157 21
leeshetnu.nl => 2 6
lekkerwonen.org => 14 25
lijstverse.nl => 4 15
likemag.com => 1 8
livekijken.nl => 2 4
martinvrijland.nl => 3 20
niburu.co => 1 1
niet100.tv => 1 6
nieuwetijdskind.com => 2 16
nieuws-dump.nl => 20 17
nieuwsblad.be => 12 40
nieuwsprimeur.wordpress.com => 10 8
nieuwstrend.nu => 1 11
not100.nl => 2 9
ongelooflijk.nu => 24 21
pauwnieuws.nl => 4 10
prankster.nl =>

In [33]:
#matrix.head()  #=> ADD CATEGORY => what's NAN??


In [34]:
#matrix.to_csv("~/disinfo-trackers-matrixout-crawl201804.csv")

# sanity checks
matrix2 = matrix.copy()
for c in matrix2:
    if c == 'category':
        continue
    matrix2[c] =    matrix2[c].apply(lambda x: 1 if x>0 else 0)
    #break

matrix2['tot_c'] = 0
matrix2['tot_j'] = 0

    
for c in matrix2:
    if c.endswith('/c'):
        matrix2['tot_c'] += matrix2[c]
    if c.endswith('/j'):
        matrix2['tot_j'] += matrix2[c]

matrix2['tot_c'] = matrix2['tot_c'].apply(lambda x: 1 if x>0 else 0)
matrix2['tot_j'] = matrix2['tot_j'].apply(lambda x: 1 if x>0 else 0)

print(matrix2.groupby('category').sum().tot_c)
print(matrix2.groupby('category').sum().tot_j)


# NL-nieuws 14 / 18 of 21 => 14 2 3.0
# NL-opinion 18 / 22 of 25 => 20 10 3.5
# NL-clickbait 53 / 56 of 61 => 55 32 5.5
# NL-pparties 7 / 18 of 25 => 15 6 2.0


category
NL-clickbait    53
NL-nieuws       14
NL-opinion      18
NL-pparties      7
Name: tot_c, dtype: int64
category
NL-clickbait    56
NL-nieuws       18
NL-opinion      22
NL-pparties     18
Name: tot_j, dtype: int64
